<a href="https://colab.research.google.com/github/Nikhil2374/OrangeWood-Lab-Yolov8-Dataset/blob/main/Type%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="mD7X5l7HIgkDK6S9QA2O")
project = rf.workspace("orangewoodtask").project("orangewood-fy1ss")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to orangewood-1 in yolov8:: 100%|██████████| 2998/2998 [00:01<00:00, 2647.93it/s]


In [ ]:
!pip install torch torchvision

In [ ]:
# Install YOLOv8 (Ultralytics package)
!pip install ultralytics

In [ ]:
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics

fatal: destination path 'ultralytics' already exists and is not an empty directory.
/content/ultralytics/ultralytics


In [ ]:
!pip install inference-sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
from inference_sdk import InferenceHTTPClient

# create an inference client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="mD7X5l7HIgkDK6S9QA2O"
)

# run inference on a local image
print(CLIENT.infer(
    "/content/orangewood-1/valid/images/IMG_2457_jpeg_jpg.rf.a4c2128f710041853556f8c2ccb33452.jpg",
    model_id="orangewood-fy1ss/2"
))

{'inference_id': '12ec6144-c5d7-4642-96c6-d4b4ea5a96a3', 'time': 0.043606502998954966, 'image': {'width': 640, 'height': 640}, 'predictions': [{'x': 348.0, 'y': 496.5, 'width': 86.0, 'height': 53.0, 'confidence': 0.9273563623428345, 'class': 'Aquarium Combined - v2 raw-1024', 'class_id': 2, 'detection_id': 'bdbac9ac-d08f-4c3c-bc26-d9a318392eab'}, {'x': 338.5, 'y': 292.5, 'width': 153.0, 'height': 39.0, 'confidence': 0.9100625514984131, 'class': 'Creatures are annotated in YOLO v5 PyTorch format.', 'class_id': 3, 'detection_id': 'c4d5036b-019b-45ec-8679-cf0addfdd3ad'}, {'x': 578.0, 'y': 341.5, 'width': 108.0, 'height': 51.0, 'confidence': 0.9062296748161316, 'class': 'Creatures are annotated in YOLO v5 PyTorch format.', 'class_id': 3, 'detection_id': 'dfd4494d-177e-4011-a81d-e40006ba4d9a'}, {'x': 599.0, 'y': 266.5, 'width': 80.0, 'height': 47.0, 'confidence': 0.8767367601394653, 'class': 'Creatures are annotated in YOLO v5 PyTorch format.', 'class_id': 3, 'detection_id': '9289fb66-b890-

In [ ]:
# Install W&B for tracking and Roboflow's inference SDK
!pip install wandb inference-sdk


In [ ]:
import wandb

# Log into W&B account
wandb.login()


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from inference_sdk import InferenceHTTPClient

# Initialize the inference client with your Roboflow API key
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",  # Roboflow inference URL
    api_key="mD7X5l7HIgkDK6S9QA2O"  # Replace with your Roboflow API key
)

# Path to the uploaded image
image_path = "/content/orangewood-1/valid/images/IMG_2424_jpeg_jpg.rf.b3b4ae8e03e16f4de5e8104fa73213fe.jpg"

# Run inference using your Roboflow model ID
response = CLIENT.infer(
    image_path,  # Path to your local image
    model_id="orangewood-fy1ss/2"  # Replace with your model ID
)


In [ ]:
# Import the necessary libraries
import wandb
import cv2
import matplotlib.pyplot as plt

# Initialize the W&B run
wandb.init(project='roboflow-evaluation', entity='nikhilkumar37641')

# Log inference results like precision, recall, and mAP to W&B
for pred in response['predictions']:
    # Assuming the predictions contain the class, confidence, and bounding box
    class_name = pred['class']
    confidence = pred['confidence']
    bbox = pred['bbox']  # Format: [x1, y1, x2, y2]

    # Log the detected class and confidence to W&B
    wandb.log({
        'class': class_name,
        'confidence': confidence,
        'bbox': bbox,
        'precision': pred.get('precision', None),  # Replace with actual precision value
        'recall': pred.get('recall', None),  # Replace with actual recall value
        'mAP50': pred.get('mAP50', None),  # Replace with mAP value if available
    })

# Optionally: Display the image with bounding boxes
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

# Draw bounding boxes on the image
for pred in response['predictions']:
    x1, y1, x2, y2 = pred['bbox']
    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

# Plot the image with bounding boxes
plt.imshow(image)
plt.axis('off')
plt.show()

# Optionally log the image to W&B
wandb.log({"predictions": wandb.Image(image, caption="Inference Result")})

# Finish the W&B run
wandb.finish()


CommError: Run initialization has timed out after 90.0 sec. 
Please refer to the documentation for additional information: https://docs.wandb.ai/guides/track/tracking-faq#initstarterror-error-communicating-with-wandb-process-

1. Are you adding any augmentations to the dataset. If yes, then which ones and why. If not, then why not?
2. How many epochs are you using for this dataset training?
3. List any other hyperparameters you are specifying while training. Give reasoning for same
4. What output metrices are you analysing. Share concrete results from those?

Answers:
(1)
I have used the foollowing ⁉Noise Augmentation: Adding noise to images (such as Gaussian or salt and pepper noise) is robust to noisy inputs; it is one of the most common techniques used in augmentations since noise can often be prevalent in real-world inputs and prevent overfitting by imparting randomness into the model.

Hue Shift: Hue shift changes the color balance of the image through adjustment in hue. It can be used to create different lighting conditions or changes in color within the environment where the model will be working.

Greyscale Augmentation: This converts the image into greyscale. Here, the color information fades away and the model will begin looking at it based on texture, shapes, and structures. This might be useful in the situation where the model has to classify based on structure rather than color.

(2)
I have used 80 epochs for the above project to increase the accuracy.

(3)
They are:
Learning Rate (lr):
Value: 1e-4
Reason: This stabilizes convergence and optimizes the optimal solution without overshooting.
Batch Size (batch_size):
Value: 16
Reason: Balances the memory occupied and gradient, thereby not overfitting. It is optimal for training.
Image Size (imgsz):
Value: 640
Reason: This is a good balance between model accuracy and training speed. It gives sufficient detail in the image while reducing the memory usage.
Momentum (momentum):
Value: 0.937
Reason: This helps accelerate training with constant smoothness of gradients and reduces oscillations.
Weight Decay (weight_decay):
Value: 0.0005
Reasoning: Adds a regularization term to the loss function, which inhibits overfitting.
Learning Rate Scheduler (lr_scheduler):

Value: 'cosine'
Reasoning: The learning rate goes down during training but improves fine-tuning and model convergence.
Anchor Boxes (anchors):

Value: YOLOv8 Default Anchors
Reasoning: Pre-defined anchors normally work perfectly on most datasets, but may be used based on object size in the dataset
IoU Threshold (iou_thres):

Value: 0.5
Reasoning: Minimum overlap between predicted and ground-truth boxes necessary for valid detection, balancing precision with recall.
These parameters will control the training and enable tuning with regard to the characteristics of the dataset as well as the performance.

(4)
Major Output Metrics to Consider are:

1. Precision:
Definition: Correct positives over the total number of predicted boxes.
Explanation: The more precise, the fewer false positives, meaning incorrect detections
Example Result: Precision = 0.85 (85% of the total predicted boxes correct).
2. Recall:
Definition: True positive bounding boxes over the actual total number of objects.
Explanation: High means there were lesser false negatives; missed detections.
Example Result: Recall = 0.78, meaning 78% of true objects are correctly detected.
3. mAP (Mean Average Precision)
mAP@0.5: The average precision at an IoU threshold of 0.5.
mAP@0.5:0.95: The average precision over multiple IoU thresholds from 0.5 through 0.95. It depicts a superior level of quantification in assessing the model's correctness.
Reasoning: These metrics summarize how the model represents the various IoU thresholds while detecting objects at different overlap levels.
Example Results:
mAP50 = 0.89 (89% average precision at IoU=0.5)
mAP50-95 = 0.72 (72% average precision across IoU thresholds from 0.5 to 0.95)
4. Loss:
Training Loss: Measures how good the model fits the training data
Validation Loss: Measures how generalizes the model towards unseen data.
Analysis: Monitoring of the loss helps analyze the model behavior whether it overfits or underfits.
Sample Output:
Training Loss = 0.08, Validation Loss = 0.12
These metrics give an overview of the accuracy of the model as well as the generalization ability. For instance, if your model has high precision with low recall, it is mostly making correct predictions but misses some objects. Conversely, if recall is high, but precision is low, the model detects most of the objects but also produces many false positives.
